In [1]:
import pandas as pd 

college_dict = {'KL': 'Agr, Consumer & Env Sciences', 
                'KY': 'Applied Health Sciences',
                'LD' : 'Armed Forces',
                'NQ' : 'Auxiliary Units',
                'LT' : 'Carle Illinois Medicine',
                'LN' : 'Center Innov in Teach Learn',
                'NA' : 'Chancellor',
                'KT' : 'College of Media',
                'NU' : 'Div Intercollegiate Athletics',
                'KW' : 'Division of General Studies',
                'KN' : 'Education',
                'MY' : 'Energy Services',
                'NN' : 'Facilities',
                'KR' : 'Fine & Applied Arts',
                'KM' : 'Gies College of Business',
                'KS' : 'Graduate College',
                'KP' : 'Grainger Engineering',
                'LQ' : 'Illinois International',
                'KU' : 'Law',
                'KV' : 'Liberal Arts & Sciences',
                'LB' : 'Medicine at UIUC',
                'NP' : 'OVCRI Institutes',
                'NS' : 'Ofc VC Inst Advancement',
                'NB' : 'Provost & VC Acad Affairs',
                'LF' : 'Public Safety',
                'LP' : 'School of Information Sciences',
                'LG' : 'School of Labor & Empl. Rel.',
                'LL' : 'School of Social Work',
                'NT' : 'Technology Services',
                'LR' : 'University Library',
                'NL' : 'VC Admin and Operations',
                'NJ' : 'VC Student Affairs',
                'NE' : 'VC for Research and Innovation',
                'LC' : 'Veterinary Medicine',
                'NV' : 'Vice Chancellor for Diversity'}

In [2]:
urlList = []
for collegeCode in college_dict.keys():
    urlList.append(f"https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/{collegeCode}.html")    

urlList

['https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/KL.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/KY.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/LD.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/NQ.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/LT.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/LN.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/NA.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/KT.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/NU.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/KW.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/KN.html',
 'https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/MY.html',
 'https://www.trustees.uilli

In [3]:
from bs4 import BeautifulSoup
import requests

def getDepts(url):
    depts_list = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    headings = soup.find_all("h3")
    for h in headings:
        if len(h.text.split('-')) == 2:
            depts_list.append(h.text)
        # this prevents tuple unpacking erros in the later loop, need to consider cleaning this later. 
        else:
            depts_list.append('N/A - N/A')
    return depts_list

print(getDepts("https://www.trustees.uillinois.edu/trustees/resources/22-23-Graybook/KL.html"))

['348 - ACES Technology Services', '521 - ADM Institute for PHL', '342 - Ag Ldrshp Educ Comm Program', '470 - Agr & Consumer Economics', '483 - Agr, Consumer & Env Sci Admn', '741 - Agricultural & Biological Engr', '538 - Animal Sciences', '384 - Cooperative Extension', '802 - Crop Sciences', '698 - Food Science & Human Nutrition', '793 - Human Dvlpmt & Family Studies', '875 - Natural Res & Env Sci', '971 - Nutritional Sciences']


In [4]:
dfs = []

x = pd.read_html(urlList[0])
for url in urlList:
  college_code = url[69:-5]
  x = pd.read_html(url)
  num_tables = len(x)
  departments_list = getDepts(url)
  for i in range(num_tables):
    df_dept = x[i]
    dept = departments_list[i]
    dept_code, dept_name = dept.split('-')
    df_dept['College Code'] = college_code
    df_dept['College Name'] = college_dict[college_code]
    df_dept['Department Code'] = dept_code.strip()
    df_dept['Department Name'] = dept_name.strip()
    dfs.append(df_dept)

df = pd.concat(dfs)
df = df.reset_index(drop=True)

In [5]:
df.to_csv("graybook-sp23.csv")